In [2]:
import os
import json

from typing import List, Dict, Any, Tuple, Optional

from formating import convert_json_to_insert_sql, convert_json_to_update_sql
from crud import insert_records, delete_records, fetch_records, get_tables

In [3]:
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
assets_dir = os.path.join(parent_dir, '_assets')
db_dir = os.path.join(parent_dir, 'fetusapp')
db_filename = 'data.sqlite'
data_filename = 'detoraki_fetusdata.json'
mapping_filename = 'column_mapping.json'

db_path = os.path.join(db_dir, db_filename)
json_file = os.path.join(assets_dir, data_filename)
mapping_file = os.path.join(current_dir, mapping_filename)

Load the source data

In [4]:
with open(json_file, 'r', encoding='utf-8') as f:
        json_data = json.load(f)

Load the column mapping from the JSON file

In [5]:
with open(mapping_file, 'r', encoding='utf-8') as f:
        column_mapping = json.load(f)

### Setup admin in Users (INSERT)

The script should already be there in _assets

In [ ]:
target_tablename = 'users'
insert_records(target_tablename, db_path)


In [ ]:
fetch_records(target_tablename, db_path)

In [ ]:
# delete_records(target_tablename, db_path)

---------------------

### Migrate Patients (INSERT)

Includes manual step to treat patients missing last_name

In [ ]:
source_tablename = 's_org_ext'
target_tablename = 'patients'
data_index = 3

sql_output = convert_json_to_insert_sql(json_data[data_index]['data'], column_mapping[source_tablename], target_tablename = target_tablename)
# print(sql_output)
output_file = os.path.join(assets_dir, f'{target_tablename}.sql')
with open(output_file, 'w', encoding='utf-8') as f:
    f.write(sql_output)

#### Data Cleansing:
* Identify records with null last name and either 
    * convert the null to the string 'null' in the generated script or
    * remove the record altogether if it doesn't have any useful payload

In [ ]:
patient_data = json_data[data_index]['data']
null_lastname_ids = [item['s_org_extID'] for item in patient_data if item['lastName'] == ""]
print(null_lastname_ids)

In [ ]:
delete_records(target_tablename, db_path)

In [ ]:
insert_records(target_tablename, db_path)

In [ ]:
fetch_records(target_tablename, db_path)

---------------------

### Migrate MedicalHistory (INSERT)

In [ ]:
source_tablename = 'tblmedhistory'
target_tablename = 'history_medical'
data_index = 10

sql_output = convert_json_to_insert_sql(json_data[data_index]['data'], column_mapping[source_tablename], target_tablename = target_tablename)
# print(sql_output)
output_file = os.path.join(assets_dir, f'{target_tablename}.sql')
with open(output_file, 'w', encoding='utf-8') as f:
    f.write(sql_output)

In [ ]:
insert_records(target_tablename, db_path)

In [ ]:
# delete_records(target_tablename, db_path)

In [ ]:
fetch_records(target_tablename, db_path)

------------------

### Migrate Short Gynaecological History data (UPDATE)

In [ ]:
source_tablename = 'tblgynhistoryshort'
target_tablename = 'history_medical'
data_index = 7

sql_output = convert_json_to_update_sql(json_data[data_index]['data'], column_mapping[source_tablename], target_tablename = target_tablename, source_id = 'MedHistoryID', target_id = 'id')
# print(sql_output)
output_file = os.path.join(assets_dir, f'{target_tablename}.sql')
with open(output_file, 'w', encoding='utf-8') as f:
    f.write(sql_output)

In [ ]:
insert_records(target_tablename, db_path)

---------

### Migrate Pathologies (UPDATE)

In [ ]:
source_tablename = 'tblpathologies'
target_tablename = 'history_medical'
data_index = 13

sql_output = convert_json_to_update_sql(json_data[data_index]['data'], column_mapping[source_tablename], target_tablename = target_tablename, source_id = 'MedHistoryID', target_id = 'id')
# print(sql_output)
output_file = os.path.join(assets_dir, f'{target_tablename}.sql')
with open(output_file, 'w', encoding='utf-8') as f:
    f.write(sql_output)

In [ ]:
insert_records(target_tablename, db_path)

In [ ]:
fetch_records(target_tablename, db_path)

----------

### Migrate ObstetricsHistory (INSERT)

In [11]:
source_tablename = 'tblobstetrichistory'
target_tablename = 'history_obstetrics'
data_index = 11 # this is the index in the json_data list where the relevant data is stored

sql_output = convert_json_to_insert_sql(json_data[data_index]['data'], column_mapping[source_tablename], target_tablename = target_tablename)
# print(sql_output)
output_file = os.path.join(assets_dir, f'{target_tablename}.sql')
with open(output_file, 'w', encoding='utf-8') as f:
    f.write(sql_output)

In [12]:
delete_records(target_tablename, db_path)

In [13]:
insert_records(target_tablename, db_path)

Executing statement: INSERT INTO history_obstetrics (id, patient_id, ft, kt, embrioulkia, te, ae, created_on, created_by, last_updated_on, last_updated_by, is_active)
VALUES
(1, 3, 2, 0, 0, 0, 0, null, 1, '2013-11-21 18:05:33', 1, 1),
(5, 66, 2, 0, 0, 3, 0, '2014-02-27 01:50:24', 1, '2014-02-27 01:50:39', 1, 1),
(6, 85, 3, 0, 0, 0, 0, '2014-05-05 14:51:40', 1, '2014-05-05 14:51:40', 1, 1),
(7, 90, 0, 0, 0, 0, 0, '2014-06-16 14:24:00', 1, '2014-06-16 14:24:00', 1, 1),
(8, 92, 2, 0, 0, 0, 1, '2014-06-16 16:56:40', 1, '2014-06-16 16:56:40', 1, 1),
(9, 93, 2, 0, 0, 1, 1, '2014-06-19 10:06:02', 1, '2014-06-19 10:06:02', 1, 1),
(10, 97, 0, 0, 0, 1, 0, '2014-07-22 17:14:06', 1, '2014-07-22 17:14:06', 1, 1),
(11, 98, 0, 0, 0, 1, 0, '2014-07-24 13:37:30', 1, '2014-07-24 13:37:30', 1, 1),
(12, 103, 2, 0, 0, 1, 0, '2014-09-16 19:14:21', 1, '2014-09-16 19:14:21', 1, 1),
(13, 110, 2, 0, 1, 0, 0, '2014-10-14 13:03:07', 1, '2014-10-14 13:03:07', 1, 1),
(14, 112, 0, 1, 0, 2, 0, '2014-10-16 16:24:23', 

In [14]:
fetch_records(target_tablename, db_path)

(1, 3, 1, 1, None, '2013-11-21 18:05:33', 1, 2, 0, 0, 0, 0)
(5, 66, 1, 1, '2014-02-27 01:50:24', '2014-02-27 01:50:39', 1, 2, 0, 0, 3, 0)
(6, 85, 1, 1, '2014-05-05 14:51:40', '2014-05-05 14:51:40', 1, 3, 0, 0, 0, 0)
(7, 90, 1, 1, '2014-06-16 14:24:00', '2014-06-16 14:24:00', 1, 0, 0, 0, 0, 0)
(8, 92, 1, 1, '2014-06-16 16:56:40', '2014-06-16 16:56:40', 1, 2, 0, 0, 0, 1)


## tblobstetrichistory_x NOT TO BE MIGRATED

### Migrate Short Pregnancies History data (INSERT)

In [16]:
source_tablename = 'tblpregnancyhistory'
target_tablename = 'pregnancy_history'
data_index = 16 # this is the index in the json_data list where the relevant data is stored

sql_output = convert_json_to_insert_sql(json_data[data_index]['data'], column_mapping[source_tablename], target_tablename = target_tablename)
# print(sql_output)
output_file = os.path.join(assets_dir, f'{target_tablename}.sql')
with open(output_file, 'w', encoding='utf-8') as f:
    f.write(sql_output)

In [21]:
insert_records(target_tablename, db_path)

Executing statement: INSERT INTO pregnancy_history (id, patient_id, ter, alcohol, smoking, amniocentesis, medication, other, diabetes, hypertension, urine_albumin, bleeding, blood_type, rhesus, hemoglobinopathies_bth, hemoglobinopathies_bs, vaginal_fluid_cultivation, mycoplasma_ureaplasma, chlamydia, herpes_hsv, hiv_12, syphilis_vdlr, hepatitis_b_bhsag, aids_hiv, red_cells, toxoplasma, cytomegalovirus_cmv, listeria, hcv, created_on, created_by, last_updated_on, last_updated_by, is_active)
VALUES
(1, 3, '1979-08-06', 'Όχι nai', 'Όχι', 'Ναι', 'Λίγα και καλά vgs', 'τι άλλα', 'όχι', 'λίγο', 'τίποτα', 'λιγουλακι', 'του μωρού?', 'αρνητικό', 'τι είναι αυτό', 'και τούτο δεν το ξέρω', 'καλή φαίνεται', 'ναι', 'όχι', 'όχι', 'όχι', 'όχι', 'όχι', 'όχι', 'αρκετά', 'όχι', 'όχι', 'όχι', 'όχι', '2013-11-09 00:00:00', 1, '2013-12-01 21:25:31', 1, 1),
(4, 3, '1976-11-18', 'Δεν πίνει', 'Δεν καπνίζει', 'σταυροβελονιά', 'σε λίγο θα τα αρχίσει', 'τι άλλα να πούμε', 'και χάρακα', 'υπερένταση θα την έλεγα', 'ν

In [22]:
fetch_records(target_tablename, db_path)

(1, 3, 1, 1, '2013-11-09 00:00:00', '2013-12-01 21:25:31', '1979-08-06', 'Όχι nai', 'Όχι', 'Ναι', 'Λίγα και καλά vgs', 'τι άλλα', 'όχι', 'λίγο', 'τίποτα', 'λιγουλακι', 'του μωρού?', 'αρνητικό', 'τι είναι αυτό', 'και τούτο δεν το ξέρω', 'καλή φαίνεται', 'ναι', 'όχι', 'όχι', 'όχι', 'όχι', 'όχι', 'όχι', 'αρκετά', 'όχι', 'όχι', 'όχι', 'όχι', 1)
(4, 3, 1, 1, '2013-11-26 00:00:00', '2013-11-30 23:05:01', '1976-11-18', 'Δεν πίνει', 'Δεν καπνίζει', 'σταυροβελονιά', 'σε λίγο θα τα αρχίσει', 'τι άλλα να πούμε', 'και χάρακα', 'υπερένταση θα την έλεγα', 'ντόινγκ', 'όχι ευτυχώς, τίποτα το πουλάκι μου', 'μηδέν', 'αρνητικό', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', 1)
(5, 3, 1, 1, None, '2014-02-22 17:32:38', '2013-08-16', None, None, None, 'ωβψωψωβψωβψωβψνδφ δσφ δσ φδ φ δσφ δσ φδσ φ σδφ δσ φ σδφδσ φ δφ δσφ δσ φ δσ φ δσφ  δσφ σδ φ δσ φ δσ φσδφ δσφ σδ φ σδφ σδ φ δσ φσ δφωβψβψωβψωβψωβψωβψωβ φδ φ φ ', None, None, None, None, None, None, None, None, None, None, None, Non

### Migrate Short Pregnancies History X data (INSERT)

In [23]:
source_tablename = 'tblpregnancyhistory_x'
target_tablename = 'pregnancy_history_x'
data_index = 17 # this is the index in the json_data list where the relevant data is stored

sql_output = convert_json_to_insert_sql(json_data[data_index]['data'], column_mapping[source_tablename], target_tablename = target_tablename)
# print(sql_output)
output_file = os.path.join(assets_dir, f'{target_tablename}.sql')
with open(output_file, 'w', encoding='utf-8') as f:
    f.write(sql_output)

In [27]:
insert_records(target_tablename, db_path)
# fetch_records(target_tablename, db_path)
# delete_records(target_tablename, db_path)

Executing statement: INSERT INTO pregnancy_history_x (id, pregnancy_id, date_of_visit, cause_of_visit, tokos, pregnancy_age, height, weight_begin, weight_current, weight_change, presentation, fetal_heart, amniotic_sac, contractions, cervical_dilation, cervical_effacement, arterial_pressure, temperature, sumphysial_fundal_height_sfh, comments, created_on, created_by, last_updated_on, last_updated_by, is_active)
VALUES
(1, 1, '2013-11-01', 'Πρώτο Ραντεβού', 1, 'μεγάλη', '1', '2', '3', '4', 'καλή και η προβολή', 'μια χαρά', 'κάνενα πρόβλημα', 'όχι ακόμη', 'δεν καταλαβαίνει', 'τώρα πάλι', 'δεν ξέρω', '36', '12', null, '2013-11-27 00:00:00', 1, '2013-11-27 00:00:00', 1, 1),
(2, 4, '2013-11-01', 'Πρώτο Ραντεβού', 1, 'μεγάλη', '1', '2', '3', '1', 'καλή και η προβολή', 'μια χαρά', 'κάνενα πρόβλημα', 'όχι ακόμη', 'δεν καταλαβαίνει', 'τώρα πάλι', 'δεν ξέρω. Να το κάνω text να το αφήσω ως έχει', '36', '12', null, '2013-11-27 00:00:00', 1, '2014-01-20 02:07:47', 1, 1),
(3, 1, '1997-01-01', 'Δεύτερ

... to be continued

In [8]:
get_tables(db_path)

('alembic_version',)
('users',)
('patients',)
('history_medical',)
('history_obstetrics',)


--------------